In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import torch
from hydra import compose, initialize
from PIL import Image 

project_root = Path().resolve().parent
sys.path.append(str(project_root))

from yolo import (
    AugmentationComposer,
    Config,
    PostProccess,
    create_converter,
    create_model,
    custom_logger,
    draw_bboxes,
)

In [ ]:
CONFIG_PATH = "../yolo/config"
CONFIG_NAME = "config"
MODEL = "v9-c"

DEVICE = 'cuda:0'
CLASS_NUM = 80
IMAGE_PATH = '../demo/images/inference/image.png'

custom_logger()
device = torch.device(DEVICE)

In [ ]:
with initialize(config_path=CONFIG_PATH, version_base=None, job_name="notebook_job"):
    cfg: Config = compose(config_name=CONFIG_NAME, overrides=["task=inference", f"task.data.source={IMAGE_PATH}", f"model={MODEL}"])
    model = create_model(cfg.model, class_num=CLASS_NUM).to(device)

    transform = AugmentationComposer([], cfg.image_size)

    converter = create_converter(cfg.model.name, model, cfg.model.anchor, cfg.image_size, device)
    post_proccess = PostProccess(converter, cfg.task.nms)

In [ ]:
pil_image = Image.open(IMAGE_PATH)
image, bbox, rev_tensor = transform(pil_image)
image = image.to(device)[None]
rev_tensor = rev_tensor.to(device)[None]

In [ ]:
with torch.no_grad():
    predict = model(image)
    pred_bbox = post_proccess(predict, rev_tensor)

draw_bboxes(pil_image, pred_bbox, idx2label=cfg.dataset.class_list)

Sample Output:

![image](../demo/images/output/visualize.png)